# Use differences as additional output

This file is for using the difference as addaitional output. We use the input voltage to predict the electrical power as well as the difference in power. We can predict the curretn difference but we need to take multiple inputs into account for a RNN to work properly.

## Preliminaries

### Import Libraries

In [1]:
# import own libraries
import Global_Functions as gf
import Neuronal_Networks as nn

# import python libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time
timestr = time.strftime("%Y-%m-%d_%H-%M_")

# import tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.callbacks import EarlyStopping

In [2]:
np.random.seed(123)

### Open files

In [3]:
OPEN_FOLDER = '../Data/Preped_Data/'

In [4]:
ex_1 = gf.open_CSV_file('experiment_1_short.csv', OPEN_FOLDER)
ex_4 = gf.open_CSV_file('experiment_4_short.csv', OPEN_FOLDER)
ex_9 = gf.open_CSV_file('experiment_9_short.csv', OPEN_FOLDER)
ex_20 = gf.open_CSV_file('experiment_20_short.csv', OPEN_FOLDER)
ex_21 = gf.open_CSV_file('experiment_21_short.csv', OPEN_FOLDER)
ex_22 = gf.open_CSV_file('experiment_22_short.csv', OPEN_FOLDER)
ex_23 = gf.open_CSV_file('experiment_23_short.csv', OPEN_FOLDER)
ex_24 = gf.open_CSV_file('experiment_24_short.csv', OPEN_FOLDER)

In [5]:
experiments = [ex_1, ex_4, ex_9, ex_20, ex_21, ex_22, ex_23, ex_24]
names = ['1', '4', '9', '20', '21','22', '23', '24']

In [6]:
#OPEN_SYNTHETIC = 'C:/Users/FlorianLeiser/Documents/Masterarbeit/Data/Synthetic_Data/Without_Noise/Sample_Ratio_1/quadratic_12_34_linear_1'

In [7]:
# experiments = gf.load_synthetic(OPEN_SYNTHETIC, length = 50)
# names = [str(x+1) for x in range(len(experiments)-1)]
# names.append("hand")

### Specify parameters

In [8]:
NEUR = 128
EPOCH = 50
LAG = 60
DIFFERENCE = "predict"

In [9]:
train = "20"
val = "21"

# train_exs = experiments[:30]
# ex_train = gf.use_multiple_experiments(train_exs)
# ex_val = gf.use_multiple_experiments(experiments[30:40])
# ex_test = gf.use_multiple_experiments(experiments[40:50])
ex_train = ex_20
ex_val = ex_21

In [10]:
image_path = "../Images/Difference/" + str(DIFFERENCE) + "/" + timestr
specs = "training_{0} val_{1} {2}_neurons__{3}_epochs__lag_{4}/" 
image_folder = image_path + specs.format(train, val, NEUR, EPOCH, LAG)
gf.check_folder(image_folder)

Creation of directory ../Images/Difference/predict/2021-05-14_18-24_training_20 val_21 128_neurons__50_epochs__lag_60/ successful.


In [11]:
# saving folder for images
model_path = "../Models/Difference/" + str(DIFFERENCE) + "/" + timestr
specs = "training_{0} val_{1} {2}_neurons__{3}_epochs__lag_{4}/" 
model_folder = model_path + specs.format(train, val, NEUR, EPOCH, LAG)

gf.check_folder(model_folder)

Creation of directory ../Models/Difference/predict/2021-05-14_18-24_training_20 val_21 128_neurons__50_epochs__lag_60/ successful.


## Training the model

In [12]:
model, history, scaler_train, X_train, y_train, _, X_val, y_val = nn.train_model(ex_train, ex_val, save_folder = model_folder,
                                                                                 difference_chosen = DIFFERENCE, lag_chosen = LAG,
                                                                                 nmb_epochs = EPOCH, neurons_chosen = NEUR)

Folder already exists.


C:\Users\FlorianLeiser\Documents\Masterarbeit\Code\Data_Processing.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['diff'] = diffs
C:\Users\FlorianLeiser\Documents\Masterarbeit\Code\Data_Processing.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['diff'] = diffs


6495/6495 [==============================] - 37s 6ms/step - loss: 0.0060 - val_loss: 0.2469
Epoch 10 done.
 803/6495 [==>...........................] - ETA: 23s - loss: 0.0051

In [ ]:
all_scaler = list()
all_X_values = list()
all_y_values = list()
all_predictions = list()
all_scaled_predictions = list()

for i in range(len(experiments)):
    scaler, X, y, preds_scaled, preds = nn.predictions(experiments[i], model,
                                                       difference_chosen = DIFFERENCE,
                                                       lag_chosen = LAG)
    all_scaler.append(scaler)
    all_X_values.append(X)
    all_y_values.append(y)
    all_predictions.append(preds)
    all_scaled_predictions.append(preds_scaled)
    
    gf.create_prediction_plot(experiments[i]['el_power'], preds[:,0], image_folder,
                              title = 'Using Difference as Output ',
                             specs = 'on ex_{0} with model trained on {1}'.format(names[i], train))
    results = gf.measure_difference(experiments[i]['el_power'], preds[:,0], should_print = False)
    results.to_csv(image_folder + "results_ex_{0}.csv".format(names[i]), sep = "|", encoding = 'utf-8')
    results_scaled = gf.measure_difference(y[:,0][:,0], preds_scaled[:,0], should_print = False)
    results_scaled.to_csv(image_folder + "results_scaled_ex_{0}.csv".format(names[i]), sep = "|", encoding = 'utf-8')
    
    gf.create_prediction_plot(y[:,0][:,1], preds_scaled[:,1], image_folder,
                              title = 'Difference in Power using Difference as Output ',
                             specs = 'differences on ex_{0} with model trained on {1}'.format(names[i], train))